In [850]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import time

In [851]:
name_file_data_report = 'Общая таблица слушателей ЦОПП.xlsx' 
path_to_end_folder_report = 'data'

In [852]:
df_dpo = pd.read_excel(name_file_data_report,sheet_name='ДПО')

In [853]:
#Создаем файл 

In [854]:
# Раздел 1.3
# Количество программ по каждому виду обучения
# группируем. Так как нам нужны текстовые данные то применяем создаем строку с помощью join
quantity_program_on_type_provisional =df_dpo.groupby('Наименование_дополнительной_профессиональной_программы').agg({'Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка':lambda x:','.join(x)})

# Применяем к полученной серии функцию разделения по запятой. Предполо
quantity_program_on_type_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка']=quantity_program_on_type_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].apply(lambda x: x.split(',')[0])

df_1_3=quantity_program_on_type_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].value_counts().to_frame()
# Раздел 1.3

In [855]:
# Создаем пустую колонку 4
temp_df = pd.DataFrame(columns=['4'])
df_1_3 = pd.concat([df_1_3,temp_df],axis=1)
# переименываем первую колонку
df_1_3.rename(columns={'Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка':'Число реализованных программ'},inplace=True)


In [856]:
group_quantity_student_program = df_dpo.groupby(['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка']).size()
group_quantity_student_program

Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка
Повышение квалификации             27
Профессиональная переподготовка     9
dtype: int64

In [857]:
# Добавляем колонку 5- всего слушателей
df_1_3['Всего слушателей'] = group_quantity_student_program
df_1_3

,Число реализованных программ,4,Всего слушателей
Повышение квалификации,2,NaN,27
Профессиональная переподготовка,1,NaN,9


In [858]:
# Подсчет сетевой формы
df_dpo_network = df_dpo[df_dpo['Использование_сетевой_формы_обучения'] == 'Сетевая форма']

In [859]:
group_quantity_network_program_provisional=df_dpo_network.groupby('Наименование_дополнительной_профессиональной_программы').agg({'Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка':lambda x:','.join(x)})
# Применяем к полученной серии функцию разделения по запятой. Предполо
group_quantity_network_program_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка']=group_quantity_network_program_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].apply(lambda x: x.split(',')[0])
df_1_3['Число программ сетевая форма'] = group_quantity_network_program_provisional['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].value_counts().to_frame()

In [860]:
df_1_3

,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма
Повышение квалификации,2,NaN,27,1
Профессиональная переподготовка,1,NaN,9,1


In [861]:
# Считаем число слушателей на сетевых программах
df_1_3['Численость слушателей сетевый программ']=df_dpo_network.groupby('Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка').agg({'ФИО_именительный':'count'})

In [862]:
df_1_3

,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма,Численость слушателей сетевый программ
Повышение квалификации,2,NaN,27,1,3
Профессиональная переподготовка,1,NaN,9,1,9


In [863]:
# Добавляем 4 пустых столбца
temp_df = pd.DataFrame(columns=['8','9','10','11','12'])
df_1_3 = pd.concat([df_1_3,temp_df],axis=1)


In [864]:
df_1_3

,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма,Численость слушателей сетевый программ,8,9,10,11,12
Повышение квалификации,2,NaN,27,1,3,NaN,NaN,NaN,NaN,NaN
Профессиональная переподготовка,1,NaN,9,1,9,NaN,NaN,NaN,NaN,NaN


In [865]:
# Считаем дистанционные технологии
df_dpo_distant = df_dpo[df_dpo['Использование_ЭО_или_ДОТ'] != 'Без применения ЭО или ДОТ']
group_quantity_distant = df_dpo_distant.groupby('Наименование_дополнительной_профессиональной_программы').agg({'Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка':lambda x:','.join(x)})

# Применяем к полученной серии функцию разделения по запятой. 
group_quantity_distant['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка']=group_quantity_distant['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].apply(lambda x: x.split(',')[0])
df_1_3['Число программ ЭО и ДОТ'] = group_quantity_distant['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'].value_counts().to_frame()


In [866]:
df_1_3

,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма,Численость слушателей сетевый программ,8,9,10,11,12,Число программ ЭО и ДОТ
Повышение квалификации,2,NaN,27,1,3,NaN,NaN,NaN,NaN,NaN,1
Профессиональная переподготовка,1,NaN,9,1,9,NaN,NaN,NaN,NaN,NaN,1


In [867]:
#Считаем количество слушателей
df_1_3['Численность слушателей ЭО И ДОТ'] = df_dpo_distant.groupby('Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка').agg({'ФИО_именительный':'count'})

In [868]:
df_1_3

,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма,Численость слушателей сетевый программ,8,9,10,11,12,Число программ ЭО и ДОТ,Численность слушателей ЭО И ДОТ
Повышение квалификации,2,NaN,27,1,3,NaN,NaN,NaN,NaN,NaN,1,3
Профессиональная переподготовка,1,NaN,9,1,9,NaN,NaN,NaN,NaN,NaN,1,9


In [869]:
# Считаем только тех кто учился на программах исключительно ЭО и ДОТ
df_dpo_distant_only_dot = df_dpo_distant[df_dpo_distant['Использование_ЭО_или_ДОТ'] == 'Исключительно с ЭО ли ДОТ' ]
df_1_3['Численность слушателей только ЭО и ДОТ'] = df_dpo_distant_only_dot.groupby('Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка').agg({'ФИО_именительный':'count'})

In [870]:
finish_df_1_3 = df_1_3.reset_index()

In [871]:
finish_df_1_3

,index,Число реализованных программ,4,Всего слушателей,Число программ сетевая форма,Численость слушателей сетевый программ,8,9,10,11,12,Число программ ЭО и ДОТ,Численность слушателей ЭО И ДОТ,Численность слушателей только ЭО и ДОТ
0,Повышение квалификации,2,NaN,27,1,3,NaN,NaN,NaN,NaN,NaN,1,3,NaN
1,Профессиональная переподготовка,1,NaN,9,1,9,NaN,NaN,NaN,NaN,NaN,1,9,NaN


In [872]:
finish_df_1_3.columns= ['Наименование образовательных программ','Число реализованных образовательных программ всего,единиц','из них прошли профессионально-общественную аккредитацию',
                'Всего слушателей, обученных по программам, человек','число программ(из графы 3) реализованных с использованием сетевой формы',
                'численность слушателей обученных(из графы 5) по программам с использованием сетевой формы-всего',
                 'в том числе(из графы 7) с использованием ресурсов иностранных организаций','общее число заключенных договоров с организациями на реализацию образовательных программ с использованием сетевой формы',
                 'из них число заключенных договоров(из графы 9) с использованием ресурсов иностранных организаций','число организаций с которыми заключены договоры  на реализацию образовательных программ с использованием сетевой формы',
                'в том числе(из графы 11) число иностранных организаций','число программ(из графы 3),реализованных с применением ЭО или ДОТ','Численность слушателей обученных(из графы 5) по программам с применением ЭО и ДОТ',
                 'в том числе(из графы 14) с применением исключительно ЭО и ДОТ']

In [873]:
finish_df_1_3

,Наименование образовательных программ,"Число реализованных образовательных программ всего,единиц",из них прошли профессионально-общественную аккредитацию,"Всего слушателей, обученных по программам, человек",число программ(из графы 3) реализованных с использованием сетевой формы,численность слушателей обученных(из графы 5) по программам с использованием сетевой формы-всего,в том числе(из графы 7) с использованием ресурсов иностранных организаций,общее число заключенных договоров с организациями на реализацию образовательных программ с использованием сетевой формы,из них число заключенных договоров(из графы 9) с использованием ресурсов иностранных организаций,число организаций с которыми заключены договоры на реализацию образовательных программ с использованием сетевой формы,в том числе(из графы 11) число иностранных организаций,"число программ(из графы 3),реализованных с применением ЭО или ДОТ",Численность слушателей обученных(из графы 5) по программам с применением ЭО и ДОТ,в том числе(из графы 14) с применением исключительно ЭО и ДОТ
0,Повышение квалификации,2,NaN,27,1,3,NaN,NaN,NaN,NaN,NaN,1,3,NaN
1,Профессиональная переподготовка,1,NaN,9,1,9,NaN,NaN,NaN,NaN,NaN,1,9,NaN


In [874]:
# Создаем файл excel
wb = openpyxl.Workbook()
# Создаем лист раздела 1.3
wb.create_sheet(title='Раздел 1.3',index=0)
wb.create_sheet(title='Раздел 2.1',index=1)
wb.create_sheet(title='Раздел 2.2',index=2)
wb.create_sheet(title='Раздел 2.3.1',index=3)
wb.create_sheet(title='Раздел 2.3.2',index=4)
wb.create_sheet(title='Раздел 2.4',index=5)
wb.create_sheet(title='Раздел 2.5',index=6)




<Worksheet "Раздел 2.5">

In [875]:
for r in dataframe_to_rows(finish_df_1_3, index=False, header=True):
    wb['Раздел 1.3'].append(r)

In [876]:
# Раздел 2.4


In [877]:
# Создаем дополнительную числовую колонку где каждое значение это 1, для удобства агрегирования
df_2_4 = df_dpo.copy()
# Добавляем колонку с 1
df_2_4['for_counting'] = 1 

In [878]:
#Считаем общее количество
all_counting_age_category=df_2_4.groupby(['Возрастная_категория_1ПК']).agg({'for_counting':'sum'}).rename(columns={'for_counting':'Всего слушателей'})

In [879]:
all_counting_age_category

,Всего слушателей
Возрастная_категория_1ПК,
30-34,4
35-39,2
40-44,7
45-49,8
50-54,5
55-59,3
60-64,6


In [880]:
df_2_4_all = all_counting_age_category.swapaxes(1,0)

In [881]:
df_2_4_all

Возрастная_категория_1ПК,30-34,35-39,40-44,45-49,50-54,55-59,60-64
Всего слушателей,4,2,7,8,5,3,6


In [882]:
# Считаем женщин
df_2_4_women=df_2_4[df_2_4['Пол_получателя']=='Жен'].groupby(['Возрастная_категория_1ПК']).agg({'for_counting':'sum'}).rename(columns={'for_counting':'Женщин Всего'}).swapaxes(1,0)

In [883]:
df_2_4_women

Возрастная_категория_1ПК,30-34,35-39,40-44,45-49,50-54,55-59,60-64
Женщин Всего,3,2,7,6,5,3,1


In [884]:
df_2_4_all =pd.concat([df_2_4_all,df_2_4_women],axis=0)

In [885]:
df_2_4_all

Возрастная_категория_1ПК,30-34,35-39,40-44,45-49,50-54,55-59,60-64
Всего слушателей,4,2,7,8,5,3,6
Женщин Всего,3,2,7,6,5,3,1


In [886]:
finish_df2_4=pd.pivot_table(df_dpo,index=['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка','Пол_получателя'],
              columns=['Возрастная_категория_1ПК'],
               values=['ФИО_именительный'],
              aggfunc='count')

In [887]:
finish_df2_4

ФИО_именительный  \
Возрастная_категория_1ПК                                                     30-34   
Дополнительная_профессиональная_программа_повыш... Пол_получателя                    
Повышение квалификации                             Жен                         2.0   
                                                   Муж                         1.0   
Профессиональная переподготовка                    Жен                         1.0   
                                                   Муж                         NaN   

                                                                               \
Возрастная_категория_1ПК                                          35-39 40-44   
Дополнительная_профессиональная_программа_повыш... Пол_получателя               
Повышение квалификации                             Жен              2.0   6.0   
                                                   Муж              NaN   NaN   
Профессиональная переподготовка                    Жен              NaN   1.0   
                                                   Муж              NaN   NaN   

                                                                               \
Возрастная_категория_1ПК                                          45-49 50-54   
Дополнительная_профессиональная_программа_повыш... Пол_получателя               
Повышение квалификации                             Жен              4.0   3.0   
                                                   Муж              2.0   NaN   
Профессиональная переподготовка                    Жен              2.0   2.0   
                                                   Муж              NaN   NaN   

                                                                               
Возрастная_категория_1ПК                                          55-59 60-64  
Дополнительная_профессиональная_программа_повыш... Пол_получателя              
Повышение квалификации                             Жен              2.0   1.0  
                                                   Муж              NaN   3.0  
Профессиональная переподготовка                    Жен              1.0   NaN  
                                                   Муж              NaN   2.0

In [888]:
finish_df2_4.columns =finish_df2_4.columns.droplevel()
finish_df2_4.index = finish_df2_4.index.droplevel()

In [889]:
finish_df2_4

Возрастная_категория_1ПК,30-34,35-39,40-44,45-49,50-54,55-59,60-64
Пол_получателя,,,,,,,
Жен,2.0,2.0,6.0,4.0,3.0,2.0,1.0
Муж,1.0,NaN,NaN,2.0,NaN,NaN,3.0
Жен,1.0,NaN,1.0,2.0,2.0,1.0,NaN
Муж,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [890]:
finish_df2_4=pd.concat([df_2_4_all,finish_df2_4])

In [891]:
finish_df2_4

Возрастная_категория_1ПК,30-34,35-39,40-44,45-49,50-54,55-59,60-64
Всего слушателей,4.0,2.0,7.0,8.0,5.0,3.0,6.0
Женщин Всего,3.0,2.0,7.0,6.0,5.0,3.0,1.0
Жен,2.0,2.0,6.0,4.0,3.0,2.0,1.0
Муж,1.0,NaN,NaN,2.0,NaN,NaN,3.0
Жен,1.0,NaN,1.0,2.0,2.0,1.0,NaN
Муж,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [892]:
# Переименовываем индекс, чтобы было более понятно
finish_df2_4.index.name = 'Наименование показателей'
finish_df2_4.index= ['Численность слушателей Всего','из них Женщин','Повышение квалификации Женщины','Повышение квалификации Мужчины',
                     'Профессиональная переподготовка Женщины','Профессиональная переподготовка Мужчины',]



In [893]:
finish_df2_4.to_excel('2.4.xlsx')

In [894]:
finish_df2_4.index

Index(['Численность слушателей Всего', 'из них Женщин',
       'Повышение квалификации Женщины', 'Повышение квалификации Мужчины',
       'Профессиональная переподготовка Женщины',
       'Профессиональная переподготовка Мужчины'],
      dtype='object')

In [895]:
#Сохраняем в лист Раздел 2.4
for r in dataframe_to_rows(finish_df2_4, index=True, header=True):
    if len(r) != 1:
        wb['Раздел 2.4'].append(r)

In [896]:
#Добавляем колонки


In [897]:
#раздел 2.5 Инвалиды
df_dpo_2_5 = df_dpo.copy()
df_dpo_2_5['for_counting'] = 1
df_dpo_2_5 = df_dpo_2_5[df_dpo_2_5['Сведения_об_ограничении_возможностей_здоровья'] != 'нет ОВЗ']



In [898]:
df_dpo_2_5

,Порядковый_номер_группы,Вид_документа,Статус_документа,Подтверждение_утраты,Подтверждение_обмена,Подтверждение_уничтожения,Серия_документа,Номер_документа,Дата_выдачи_документа,Регистрационный_номер,...,Использование_сетевой_формы_обучения,Использование_ЭО_или_ДОТ,Модульная_программа_с_вариативным_выбором,Вид_экономической\nдеятельности_дополнительной_профессиональной_программы,Для_освоения_ДПП_требуется_наличие_ВО,Номер_приказа_о_зачислении,Номер_приказа_об_отчислении,Текущий_возраст,Возрастная_категория_1ПК,for_counting
0,NaN,Удостоверение о повышении квалификации,Оригинал,Нет,Нет,Нет,нет,32413862269,2022-01-28,768,...,Нет,Без применения ЭО или ДОТ,да,образование,требуется ВО,3,4,45,45-49,1
1,NaN,Удостоверение о повышении квалификации,Оригинал,Нет,Нет,Нет,нет,32413862270,2022-01-28,769,...,Нет,Без применения ЭО или ДОТ,да,образование,требуется ВО,3,4,60,60-64,1
34,NaN,Диплом о профессиональной переподготовке,Оригинал,Нет,Нет,Нет,нет,32413862276,2022-01-28,775,...,Сетевая форма,С применением ЭО или ДОТ,нет,образование,требуется ВО,3,4,41,40-44,1
35,NaN,Диплом о профессиональной переподготовке,Оригинал,Нет,Нет,Нет,нет,32413862277,2022-01-28,776,...,Сетевая форма,С применением ЭО или ДОТ,нет,образование,требуется ВО,3,4,63,60-64,1


In [899]:
df_2_5_all = df_dpo_2_5.groupby(['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка']).agg({'ФИО_именительный':'count'}).rename(columns={'ФИО_именительный':'Всего'}).swapaxes(1,0)

In [900]:
finish_df2_5=pd.pivot_table(df_dpo_2_5,
              index=['Сведения_об_ограничении_возможностей_здоровья'],
              columns=['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка'],
              values=['for_counting'],
              aggfunc='sum')

In [901]:
finish_df2_5


for_counting  \
Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка Повышение квалификации   
Сведения_об_ограничении_возможностей_здоровья                                                                             
инвалид                                                                                                               1   
лицо с ОВЗ                                                                                                            1   

                                                                                                                                  
Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка Профессиональная переподготовка  
Сведения_об_ограничении_возможностей_здоровья                                                                                     
инвалид                                                                                                                        1  
лицо с ОВЗ                                                                                                                     1

In [902]:
finish_df2_5.columns = finish_df2_5.columns.droplevel(0)

In [903]:
# Соединяем
finish_df2_5 = pd.concat([df_2_5_all,finish_df2_5])

In [904]:
finish_df2_5

Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка,Повышение квалификации,Профессиональная переподготовка
Всего,2,2
инвалид,1,1
лицо с ОВЗ,1,1


In [905]:
# заполняем пропуски нулями
finish_df2_5.fillna(0,inplace=True)

In [906]:
# Необходимо как то проверять наличие колонок, ведь не факт что в таблице будет повышение квалификации или проф подготовка
#создаем колонку всего

#Проверяем количество колонок
# Заполняем нулями чтобы корректно слаживалось
if len(finish_df2_5.columns) == 1:
    finish_df2_5['Всего слушателей по всем категориям'] = finish_df2_5.iloc[:,0]
elif len(finish_df2_5.columns):
    finish_df2_5['Всего слушателей по всем категориям'] = finish_df2_5.iloc[:,0] + finish_df2_5.iloc[:,1]

In [907]:
finish_df2_5.index

Index(['Всего', 'инвалид', 'лицо с ОВЗ'], dtype='object')

In [908]:
finish_df2_5

Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка,Повышение квалификации,Профессиональная переподготовка,Всего слушателей по всем категориям
Всего,2,2,4
инвалид,1,1,2
лицо с ОВЗ,1,1,2


In [909]:
# Считаем колонку с женщинами инвалидами
df_2_5_women =df_dpo_2_5[df_dpo_2_5['Пол_получателя']=='Жен'].groupby('Сведения_об_ограничении_возможностей_здоровья').agg({'for_counting':'sum'})

In [910]:
df_2_5_women.columns =['Количество женщин']


In [911]:
sum_woman = df_2_5_women['Количество женщин'].sum()

if sum_woman == 0:
    # т.е если женщин инвалидов нет, то создаем датафрейм с пустыми значениями
    df_2_5_women_all = pd.DataFrame(data=[np.NaN,np.NaN,np.NaN],index=['Всего','инвалид','Лицо с ОВЗ'],columns=['Количество женщин'])
else:
    # в противном случае, считаем общее количество женщин-инвалидов. Создаем маленький датафрейм 
    df_2_5_women_all = pd.DataFrame(data=sum_woman,index=['Всего женщин инвалидов'],columns=['Количество женщин'])
    # Объединяем столбы с женщинами инвалидами
    df_2_5_women_all = pd.concat([df_2_5_women_all,df_2_5_women])
    
    
    

In [912]:
finish_df2_5 = finish_df2_5.astype(float)

In [913]:
finish_df2_5 = finish_df2_5.replace(0.0,np.NaN)

In [914]:
finish_df2_5

Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка,Повышение квалификации,Профессиональная переподготовка,Всего слушателей по всем категориям
Всего,2.0,2.0,4.0
инвалид,1.0,1.0,2.0
лицо с ОВЗ,1.0,1.0,2.0


In [915]:
finish_df2_5.to_excel('2.5.xlsx')

In [916]:
#Сохраняем в лист Раздел 2.5
for r in dataframe_to_rows(finish_df2_5, index=True, header=True):
    if len(r) != 1:
        wb['Раздел 2.5'].append(r)

In [917]:
#Добавляем в раздел 2.5 информацию о женщинах
for r in dataframe_to_rows(df_2_5_women_all, index=True, header=True):
    if len(r) != 1:
        wb['Раздел 2.5'].append(r)

In [918]:
 # Получаем текущее время для того чтобы использовать в названии
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder_report}/Часть отчета 1-ПК.xlsx {current_time}.xlsx')
